In [288]:
import datetime
import gspread
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from urllib.request import urlopen
import time
from basketball_reference_scraper.pbp import get_pbp

pd.options.mode.chained_assignment = None  # default='warn'


In [289]:
date = '2022-11-19'
away_team = 'PHI'
home_team = 'MIN'

In [290]:
pbp_df = get_pbp(date, away_team, home_team)
pbp_df = pbp_df.rename(columns={
    pbp_df.columns[4]:"AWAY_SCORE",
    pbp_df.columns[5]:"HOME_SCORE"
})

pbp_df = pbp_df[['QUARTER','TIME_REMAINING','AWAY_SCORE','HOME_SCORE']]
pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'])
pbp_df['TIME_REMAINING'] = pd.to_datetime(pbp_df['TIME_REMAINING'],format= '%H:%M:%S' ).dt.time
pbp_df['AWAY_SCORE'] = pbp_df['AWAY_SCORE'].astype(int)
pbp_df['HOME_SCORE'] = pbp_df['HOME_SCORE'].astype(int)

quarter_beginning = '12:00:00'
quarter_ending = '00:00:00'
overtime_beginning = '05:00:00'

list_of_quarters = list(pbp_df['QUARTER'].unique())

full_df = pd.DataFrame()
for quarter in list_of_quarters:
    quarter_df = pbp_df[pbp_df['QUARTER'] == quarter]

    if quarter == 1:# add beginning of 1st quarter
        quarter_df.loc[-1] = [1, quarter_beginning, 0, 0] 
        quarter_df.index = quarter_df.index + 1  
        quarter_df = quarter_df.sort_index()

        end_row = {
            'QUARTER': [quarter],
            'TIME_REMAINING': quarter_ending,
            'AWAY_SCORE': [quarter_df.iloc[-1,2]],
            'HOME_SCORE': [quarter_df.iloc[-1,3]]
        }

        end_row = pd.DataFrame(data=end_row)
        quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)
        
        temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
        quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
        quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
        quarter_df['INDICATOR'] = np.where(
            quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
            ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
            0
        )
        
        full_df = pd.concat([full_df, quarter_df], axis=0)


    elif quarter in (2,3,4):
        start_row = end_row
        start_row['QUARTER'] = quarter
        start_row['TIME_REMAINING'] = quarter_beginning

        quarter_df = quarter_df.reset_index(drop=True)
        quarter_df.index += 1

        quarter_df = pd.concat([start_row, quarter_df],axis=0)
        quarter_df = quarter_df.sort_index()

        end_row = {
            'QUARTER': [quarter],
            'TIME_REMAINING': quarter_ending,
            'AWAY_SCORE': [q_df.iloc[-1,2]],
            'HOME_SCORE': [q_df.iloc[-1,3]]
        }

        end_row = pd.DataFrame(data=end_row)
        quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

        temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
        quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
        quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
        quarter_df['INDICATOR'] = np.where(
            quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
            ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
            0
        )
        
        full_df = pd.concat([full_df, quarter_df], axis=0)
        
    else:
        start_row = end_row
        start_row['QUARTER'] = quarter
        start_row['TIME_REMAINING'] = overtime_beginning

        quarter_df = quarter_df.reset_index(drop=True)
        quarter_df.index += 1

        quarter_df = pd.concat([start_row, quarter_df],axis=0)
        quarter_df = quarter_df.sort_index()

        end_row = {
            'QUARTER': [quarter],
            'TIME_REMAINING': quarter_ending,
            'AWAY_SCORE': [quarter_df.iloc[-1,2]],
            'HOME_SCORE': [quarter_df.iloc[-1,3]]
        }

        end_row = pd.DataFrame(data=end_row)
        quarter_df = pd.concat([quarter_df, end_row], axis=0).reset_index(drop=True)

        temp_date = str(datetime.datetime.strptime('1900-01-01', '%Y-%m-%d').date())
        quarter_df['TIME_REMAINING'] = pd.to_datetime(temp_date + " " + quarter_df.TIME_REMAINING.astype(str))
        quarter_df['TIME_ELAPSED'] = (quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))
        quarter_df['INDICATOR'] = np.where(
            quarter_df['QUARTER'] == quarter_df['QUARTER'].shift(-1),
            ((quarter_df['TIME_REMAINING'] - quarter_df['TIME_REMAINING'].shift(-1))/60000000000).view(int),
            0
        )
        
        full_df = pd.concat([full_df, quarter_df], axis=0)

full_df['LEADER'] = np.where(
    full_df.HOME_SCORE > full_df.AWAY_SCORE,
    'HOME_TEAM',
    'AWAY_TEAM',
)

full_df['LEADER'] = np.where(
    full_df.HOME_SCORE == full_df.AWAY_SCORE,
    'NEITHER',
    full_df.LEADER
)
full_df_gb = full_df.groupby('LEADER')['INDICATOR'].sum().reset_index() 

teams = ['AWAY_TEAM','HOME_TEAM']
total_seconds = full_df_gb.INDICATOR.sum()
results_dict = {}

for team in teams:
    seconds_leading = full_df_gb[full_df_gb.LEADER == team].INDICATOR.values[0]
    results_dict[team] = seconds_leading / total_seconds

results_dict
    

{'AWAY_TEAM': 0.8090277777777778, 'HOME_TEAM': 0.12881944444444443}